In [1]:
import findspark
findspark.init()
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
#REPORTE DE LOS 3 MESES CON MAS VENTAS

In [2]:
#Leemos el archivo este ya sin encabezado y "inferschema=true" nos da los tipos de datos desde su lectura
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\breyn\\Desktop\\PySparkPruebas\\Sales.csv")
rddfiltro = text_file.rdd.map(tuple)
#Se toma columna 1 y 2 que es la fecha y el total
rddMES = rddfiltro.map(lambda word: (word[1].split('/')[1],word[2]))
rddMES.take(10)

[('05', 1121.2),
 ('05', 18.5),
 ('05', 160.0),
 ('05', 397.5),
 ('05', 22.0),
 ('05', 1314.19),
 ('05', 73.0),
 ('05', 845.49),
 ('05', 85.7),
 ('05', 200.2)]

In [3]:
#Conteo total por Mes
rddCONTEO=rddMES.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONTEO.collect())

Conteo total -> [('05', 2790276.789999995), ('06', 1686344.9400000002), ('07', 242886.1100000001), ('08', 1618355.7199999967)]


In [4]:
#Ordenamiento por valor descendentemente(False en sortBy) y se toman los primeros 3
rddORDEN = sc.parallelize(rddCONTEO.sortBy(lambda a: a[1],False).take(3))
print("Meses con mayor venta -> %s" % rddORDEN.collect())

Meses con mayor venta -> [('05', 2790276.789999995), ('06', 1686344.9400000002), ('08', 1618355.7199999967)]


In [5]:
#Separando nombres y totales
#Vista de datos
rddNombres = rddORDEN.map(lambda x: (x[0]))
rddTotales = rddORDEN.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

['05', '06', '08']
[2790276.789999995, 1686344.9400000002, 1618355.7199999967]


In [6]:
#Graficando pie
fig = go.Figure(data=go.Pie(labels=rddNombres.collect(),values=rddTotales.collect()))
fig.update_layout(title_text='3 primeros meses con mayor venta.',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20)
fig.write_html('Reporte4.html', auto_open=True)

In [8]:
#REPORTE DE LOS 5 CAJEROS CON MENOS VENTAS

In [7]:
#Se toma columna 4 y 2 que es el cajero y el total
rddCAJERO = rddfiltro.map(lambda word: (word[4],word[2]))
rddCAJERO.take(10)

[('MAYTE', 1121.2),
 ('MAYTE', 18.5),
 ('MAYTE', 160.0),
 ('MAYTE', 397.5),
 ('MAYTE', 22.0),
 ('MAYTE', 1314.19),
 ('MAYTE', 73.0),
 ('MAYTE', 845.49),
 ('MAYTE', 85.7),
 ('MAYTE', 200.2)]

In [8]:
#Conteo total por Cajero
rddCONTEO=rddCAJERO.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONTEO.collect())

Conteo total -> [('MAYTE', 379808.1899999996), ('GABY LUCIO', 424765.1200000003), ('ALE HUERTA', 989779.0999999971), ('FERNANDO C', 796860.5099999999), ('MARICRUZ', 907778.1399999997), ('EDUARDO V', 114529.07999999999), ('MONSE L.', 176139.10999999993), ('SARAHY', 98553.36999999994), ('JANET', 561988.7100000003), ('JUAN MARTIN', 717881.1600000005), ('CLAUDIA', 21501.340000000004), ('LINA', 81246.10999999997), ('ALE DIAZ', 94310.20999999999), ('MAGO', 9969.440000000002), ('ELI C.', 38652.409999999996), ('JAQUELINE', 58254.79000000001), ('ROSI MEJIA', 344310.13000000035), ('JESUS PEREZ', 310391.2900000002), ('MIRIAM', 190878.87000000002), ('EDUARDO', 20266.480000000003)]


In [9]:
#Ordenamiento por valor ascendentemente(True en sortBy) y se toman los primeros 5
rddORDEN = sc.parallelize(rddCONTEO.sortBy(lambda a: a[1],True).take(5))
print("Cajeros con menor venta -> %s" % rddORDEN.collect())

Cajeros con menor venta -> [('MAGO', 9969.440000000002), ('EDUARDO', 20266.480000000003), ('CLAUDIA', 21501.340000000004), ('ELI C.', 38652.409999999996), ('JAQUELINE', 58254.79000000001)]


In [10]:
#Separando nombres y totales
#Vista de datos
rddNombres = rddORDEN.map(lambda x: (x[0]))
rddTotales = rddORDEN.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

['MAGO', 'EDUARDO', 'CLAUDIA', 'ELI C.', 'JAQUELINE']
[9969.440000000002, 20266.480000000003, 21501.340000000004, 38652.409999999996, 58254.79000000001]


In [11]:
#Graficando barras
fig = go.Figure(data=go.Bar(x=rddNombres.collect(),y=rddTotales.collect()))
fig.update_layout(title_text='Top 5 Cajeros con menor ventas.',title_font_size=30,
                  yaxis=dict(title='Total de venta',title_font_size=25),
                  xaxis=dict(title='Nombre Cajero',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('Reporte5.html', auto_open=True)